# Financial Markets - Lesson 7

In this lesson we're going to:

* go through the new additions to finmarkets.py;
* implement a credit curve class;
* if there's time, start talking about credit default swaps.

## New additions to finmarkets.py

In [1]:
from curve_data import discount_curve, libor_curve, pricing_date

In [2]:
from finmarkets import InterestRateSwap, InterestRateSwaption

In [3]:
irs = InterestRateSwap(pricing_date, 1e6, 0.01, 6, 4)
irs.npv(discount_curve, libor_curve)

-34969.02685005371

In [4]:
from datetime import date
swaption = InterestRateSwaption(date(2017, 6, 1), irs)
swaption.npv_bs(discount_curve, libor_curve, 0.07)

0.0

In [5]:
swaption.npv_mc(discount_curve, libor_curve, 0.07)

(0.0, 0.0)

## Credit curves

Just like a discount curve is an abstract way of representing the underlying interest rate implicit in the market quotes of a collection of real-world interest rate products, **credit curves** are a way of representing tha data implied by credit default swaps.

**Credit default swaps (CDS)** are instruments whose value depends on the likelihood that a given company (the curve's issuer) will suffer a credit event over a given period.

A credit event can be a default, the failure to make payments, the issuer entering into bankruptcy proceedings, or the occurence of other legal events. The exact definition of what constitutes a credit event depends on a series of factors and is usually defined in some kind of ISDA master agreement, or the event can be declared by ISDA (International Swaps and Derivatives Association) itself.

In any case, we will generically call a credit event a 'default', and talk about **non-default probabilities (NDP)**, i.e. the probability that the issuer will not suffer a credit event before a given value date. NDPs are the equivalent for credit curve of discount factors for discount curves. Just like discount curves, credit curves are built by specifying a pricing/observation date, a sequence of pillar dates and a sequence of pillar NDPs, and provide a method which log-linearly interpolates these pillar NDPs to return the NDP at an arbitrary value date between the pricing date and the last pillar date.

In addition, we'll also write a method which returns tha **hazard rate** at an arbitray value date. The hazard rate is the credit curve equivalent of the short rate or overnight rate for discount curves. It represents the instantaneous probability of the issuer defaulting conditioned on it not having defaulted until that moment - though in practice we'll calculate it numerically, and therfore it'll be the (annualized) conditional probability of the issuer defaulting between the value date and the day after.

In [7]:
import math, numpy

class CreditCurve():
    def __init__(self, pillar_dates, pillar_ndps):
        self.pillar_dates = pillar_dates
        self.pillar_ndps = pillar_ndps
        
        self.pillar_days = [(pd - pillar_dates[0]).days for pd in pillar_dates]
        self.log_ndps = [math.log(ndp) for ndp in pillar_ndps]

    def ndp(self, value_date):
        value_days = (value_date - self.pillar_dates[0]).days
        return math.exp(numpy.interp(value_days, self.pillar_days, self.log_ndps))
    
    def hazard(self, value_date):
        ndp_1 = self.ndp(value_date)
        ndp_2 = self.ndp(value_date + relativedelta(days=1))
        delta_t = 1.0 / 365.0
        h = -1.0 / ndp_1 * (ndp_2 - ndp_1) / delta_t
        return h

In [9]:
from datetime import date
from dateutil.relativedelta import relativedelta

cc = CreditCurve([pricing_date, pricing_date + relativedelta(years=2)], [1.0, 0.8])

In [10]:
cc.ndp(pricing_date + relativedelta(years=1))

0.8944271909999159

In [11]:
cc.hazard(pricing_date + relativedelta(years=1))

0.11155472498198307